In [ ]:
from __future__ import print_function
import os,sys
import ROOT as rt
from larlite import larlite
from ROOT import larutil
import chart_studio.plotly as py
import plotly.graph_objects as go
import numpy as np
import lardly

In [ ]:
# LOAD THE FILE
triplet_files = ["test.root"]
tree = rt.TChain("larmatchtriplet")
for f in triplet_files:
    tree.AddFile( f )
nentries = tree.GetEntries()
print("Num entries: ",nentries)

In [ ]:
# SETUP THE GEOMETRY

# Set the detector in larlite
detid = larlite.geo.kICARUS
#detid = larlite.geo.kMicroBooNE
#detid = larlite.geo.kSBND
larutil.LArUtilConfig.SetDetector(detid)

# Get geometry class
geo = larlite.larutil.Geometry.GetME()

# Get detector outlines from lardly
from lardly.detectors.getdetectoroutlines import DetectorOutlineFromLarlite
detoutline = DetectorOutlineFromLarlite(detid)
detlines = detoutline.getlines()

In [ ]:
# Get entry data
ENTRY = 0
tree.GetEntry(ENTRY)

spacepoint_arrays = []
for imatchdata in range(tree.triplet_v.size()):
    matchdata = tree.triplet_v.at(imatchdata)
    ntriplets = matchdata._triplet_v.size() # needs function
    if ntriplets==0:
        continue
    icryo = matchdata._trip_cryo_tpc_v.at(0).at(0) # needs function
    itpc  = matchdata._trip_cryo_tpc_v.at(0).at(1) # needs function
    
    spdata = matchdata.make_spacepoint_charge_array()
    print(spdata['spacepoint_t'].shape)
    spacepoint_arrays.append( (icryo,itpc,spdata["spacepoint_t"]) )

In [ ]:

sp_plots = []
for (icryo,itpc,pos) in spacepoint_arrays:
    q = np.clip( np.average( pos[:,3:], axis=1 ), 0, 100 )
    plot = go.Scatter3d( x=pos[:,0], y=pos[:,1], z=pos[:,2], 
                        mode="markers", name="(%d,%d)"%(icryo,itpc), 
                        marker={"size":0.5,"opacity":0.1,"color":q,"colorscale":"Viridis"} )  
    sp_plots.append(plot)

axis_template = {
    "showbackground": True,
    "backgroundcolor": "rgba(100, 100, 100,0.5)",
    "gridcolor": "rgb(50, 50, 50)",
    "zerolinecolor": "rgb(0, 0, 0)",
}


layout = go.Layout(
    title='DETECTOR TPC',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    scene= {
        "xaxis": axis_template,
        "yaxis": axis_template,
        "zaxis": axis_template,
        "aspectratio": {"x": 2, "y": 1, "z": 2},
        "camera": {"eye": {"x": -3, "y": 0.1, "z": 0.0},
                   "center":dict(x=0, y=0, z=0),
                   "up":dict(x=0, y=1, z=0)},
        "annotations": [],
    }
)

fig = go.Figure(data=sp_plots+detlines, layout=layout)
fig.show()
 